<a href="https://colab.research.google.com/github/noahdanieldsouza/PAM-classification/blob/main/ClusterClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Imports. { vertical-output: true }
!pip install git+https://github.com/google-research/perch-hoplite.git@782acd0e409eb27df51a695de4cb6608dae0db25


  Cloning https://github.com/google-research/perch-hoplite.git (to revision 782acd0e409eb27df51a695de4cb6608dae0db25) to /tmp/pip-req-build-wqfm52d8
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/perch-hoplite.git /tmp/pip-req-build-wqfm52d8
  Running command git rev-parse -q --verify 'sha^782acd0e409eb27df51a695de4cb6608dae0db25'
  Running command git fetch -q https://github.com/google-research/perch-hoplite.git 782acd0e409eb27df51a695de4cb6608dae0db25
  Running command git checkout -q 782acd0e409eb27df51a695de4cb6608dae0db25
  Resolved https://github.com/google-research/perch-hoplite.git to commit 782acd0e409eb27df51a695de4cb6608dae0db25
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61

In [ ]:
import os

from matplotlib import pyplot as plt
import numpy as np

from perch_hoplite.agile import audio_loader
from perch_hoplite.agile import classifier
from perch_hoplite.agile import classifier_data
from perch_hoplite.agile import embedding_display
from perch_hoplite.agile import source_info
from perch_hoplite.db  import brutalism
from perch_hoplite.db import score_functions
from perch_hoplite.db  import search_results
from perch_hoplite.db import sqlite_usearch_impl
from perch_hoplite.zoo import model_configs

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Correct way to copy a folder:
!cp -r /content/drive/MyDrive/test /content

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#@title Load model and connect to database. { vertical-output: true }

#@markdown Location of database containing audio embeddings.
db_path = '/content/drive/My Drive/classifier_training/DB'  #@param {type:'string'}
#@markdown Identifier (eg, name) to attach to labels produced during validation.
annotator_id = 'linnaeus'  #@param {type:'string'}

db = sqlite_usearch_impl.SQLiteUsearchDB.create(db_path)
db_model_config = db.get_metadata('model_config')

#from perch_hoplite.zoo import hub

embed_config = db.get_metadata('audio_sources')
model_class = model_configs.get_model_class(db_model_config.model_key)
embedding_model = model_class.from_config(db_model_config.model_config)
audio_sources = source_info.AudioSources.from_config_dict(embed_config)
if hasattr(embedding_model, 'window_size_s'):
  window_size_s = embedding_model.window_size_s
else:
  window_size_s = 5.0
audio_filepath_loader = audio_loader.make_filepath_loader(
    audio_sources=audio_sources,
    window_size_s=window_size_s,
    sample_rate_hz=embedding_model.sample_rate,
)
print ("Model config:", db_model_config)
print("Embeddings in DB:", len(db.get_embedding_ids()))
print(db.get_classes())

Mounted at /content/drive
Model config: embedding_dim: 1280
model_config:
  hop_size_s: 5.0
  model_path: ''
  sample_rate: 32000
  tfhub_version: 8
  window_size_s: 5.0
model_key: taxonomy_model_tf

Embeddings in DB: 33046
('boat', 'non_boat')


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#@title Load query audio. { vertical-output: true }

#@markdown The `query_uri` can be a URL, filepath, or Xeno-Canto ID
#@markdown (like `xc777802`, containing an Eastern Whipbird (`easwhi1`)).
query_uri = '/content/drive/MyDrive/full_labeled_fish/burbot3_000020_000025.wav' #@param {type:'string'}
query_label = 'sample'  #@param {type:'string'}

query = embedding_display.QueryDisplay(
    uri=query_uri, offset_s=0.0, window_size_s=5.0, sample_rate_hz=32000)
_ = query.display_interactive()

Mounted at /content/drive


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='offset_s', max=5.0), Output…

In [ ]:
query_embedding = embedding_model.embed(
    query.get_audio_window()).embeddings[0, 0]

In [ ]:
import joblib
import numpy as np
import hdbscan

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ----------------------------
# Load saved model
# ----------------------------
model = joblib.load("/content/drive/MyDrive/umap_clusterer_model.joblib")
reducer = model["umap"]
clusterer = model["clusterer"]

# ----------------------------
# New embedding
# ----------------------------
embedding = np.array(query_embedding)  # shape (embedding_dim,)
embedding = embedding.reshape(1, -1)

# ----------------------------
# Project into UMAP space
# ----------------------------
Z = reducer.transform(embedding)

# ----------------------------
# Predict cluster
# ----------------------------
labels, strengths = hdbscan.approximate_predict(clusterer, Z)

cluster_id = labels[0]
confidence = strengths[0]

print("Cluster:", cluster_id)
print("Confidence:", confidence)


Mounted at /content/drive
Cluster: 3
Confidence: 1.0
